# Configurações Iniciais para treinamento de modelo

In [8]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from xgboost import XGBRegressor
import mlflow
import mlflow.sklearn
import os
import joblib
from pathlib import Path
import sys
import logging
from datetime import datetime
from fiap import LoggerManager

# Defina o caminho absoluto onde quer salvar os mlruns
print(f'Mlflow path: {mlflow.get_tracking_uri()}')
mlflow_path = Path.cwd().parent / 'mlruns'
# Cria a pasta se não existir
os.makedirs(mlflow_path, exist_ok=True)

# Diz pro MLflow usar esse caminho
mlflow.set_tracking_uri(f'file:///{mlflow_path}')

print(f'MLflow tracking URI configurada para: {mlflow.get_tracking_uri()}')

RANDOM_STATE = 345

# Adiciona o diretório src ao path do Python
src_path = Path.cwd().parent / 'src'
if str(src_path) not in sys.path:
	sys.path.insert(0, str(src_path))

# Logger customizado
log_path = Path.cwd().parent / 'logs'
LoggerManager(log_path=log_path, base_filename='model_train')

logging.info('=' * 50)
logging.info(f'Início do treinamento: {datetime.now()}')

2026-02-28 21:56:54,098 [INFO] [G:\Meu Drive\pascon_ofc\_fiap_tech_challenges\tech_challenge_fase5\src\fiap\utils\logger_manager.py:130:__init__] Logger initialized: G:\Meu Drive\pascon_ofc\_fiap_tech_challenges\tech_challenge_fase5\logs\2026-03-01_model_train.json
2026-02-28 21:56:54,098 [INFO] [C:\Users\DELL\AppData\Local\Temp\ipykernel_17400\2645291713.py:43:<module>] ==================================================
2026-02-28 21:56:54,106 [INFO] [C:\Users\DELL\AppData\Local\Temp\ipykernel_17400\2645291713.py:44:<module>] Início do treinamento: 2026-02-28 21:56:54.106021


Mlflow path: file:///g:\Meu Drive\pascon_ofc\_fiap_tech_challenges\tech_challenge_fase5\mlruns
MLflow tracking URI configurada para: file:///g:\Meu Drive\pascon_ofc\_fiap_tech_challenges\tech_challenge_fase5\mlruns


# Carregar dados

In [9]:
logging.info('Carregar arquivo de dados processados')
df = pd.read_csv(Path.cwd().parent / 'data' / 'processed_data.csv')
logging.info(f'Dados carregados: {df.shape[0]} linhas, {df.shape[1]} colunas')

logging.info('Verificar tipos e valores nulos')
logging.info(f'{df.dtypes}')
logging.info(f'Valores nulos por coluna:\n{df.isnull().sum()}')

2026-02-28 21:56:54,125 [INFO] [C:\Users\DELL\AppData\Local\Temp\ipykernel_17400\487534782.py:1:<module>] Carregar arquivo de dados processados
2026-02-28 21:56:54,243 [INFO] [C:\Users\DELL\AppData\Local\Temp\ipykernel_17400\487534782.py:3:<module>] Dados carregados: 1156 linhas, 20 colunas
2026-02-28 21:56:54,243 [INFO] [C:\Users\DELL\AppData\Local\Temp\ipykernel_17400\487534782.py:5:<module>] Verificar tipos e valores nulos
2026-02-28 21:56:54,243 [INFO] [C:\Users\DELL\AppData\Local\Temp\ipykernel_17400\487534782.py:6:<module>] fase                    int64
idade                   int64
iaa                   float64
ieg                   float64
ips                   float64
ipp                   float64
ida                   float64
mat                   float64
por                   float64
ipv                   float64
defasagem               int64
genero_f                int64
genero_m                int64
instituição_tipo_1      int64
instituição_tipo_2      int64
instituição_ti

# Separar e normalizar dados

In [10]:
X = df.drop(columns=['defasagem'])
y = df['defasagem']

# =====================================================
# Normalização
# =====================================================
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
logging.info('Dados normalizados com MinMaxScaler')
# salvar o scaler para uso futuro
joblib.dump(scaler, Path.cwd().parent / 'ml_models' / 'scaler.joblib')
logging.info(f"Scaler salvo em: {Path.cwd().parent / 'ml_models' / 'scaler.joblib'}")

2026-02-28 21:56:54,307 [INFO] [C:\Users\DELL\AppData\Local\Temp\ipykernel_17400\1210121374.py:9:<module>] Dados normalizados com MinMaxScaler
2026-02-28 21:56:54,363 [INFO] [C:\Users\DELL\AppData\Local\Temp\ipykernel_17400\1210121374.py:12:<module>] Scaler salvo em: g:\Meu Drive\pascon_ofc\_fiap_tech_challenges\tech_challenge_fase5\ml_models\scaler.joblib


In [11]:
from fiap.utils.model_train import log_extreme_examples

log_extreme_examples(y, X)

2026-02-28 21:56:54,379 [INFO] [G:\Meu Drive\pascon_ofc\_fiap_tech_challenges\tech_challenge_fase5\src\fiap\utils\model_train.py:17:log_extreme_examples] Aluno com defasagem -2: X={'fase': 0.0, 'idade': 10.0, 'iaa': 9.002, 'ieg': 8.6136363635, 'ips': 7.51, 'ipp': 7.1875, 'ida': 7.5, 'mat': 10.0, 'por': 5.0, 'ipv': 6.27, 'genero_f': 0.0, 'genero_m': 1.0, 'instituição_tipo_1': 1.0, 'instituição_tipo_2': 0.0, 'instituição_tipo_3': 0.0, 'instituição_tipo_4': 0.0, 'instituição_tipo_5': 0.0, 'instituição_tipo_6': 0.0, 'instituição_tipo_7': 0.0}, y=-2
2026-02-28 21:56:54,382 [INFO] [G:\Meu Drive\pascon_ofc\_fiap_tech_challenges\tech_challenge_fase5\src\fiap\utils\model_train.py:17:log_extreme_examples] Aluno com defasagem 2: X={'fase': 5.0, 'idade': 14.0, 'iaa': 9.168, 'ieg': 9.431818181666666, 'ips': 7.51, 'ipp': 8.125, 'ida': 8.5, 'mat': 9.0, 'por': 8.5, 'ipv': 8.674, 'genero_f': 0.0, 'genero_m': 1.0, 'instituição_tipo_1': 0.0, 'instituição_tipo_2': 1.0, 'instituição_tipo_3': 0.0, 'institui

# Separar em treino e teste

In [12]:
X_train, X_test, y_train, y_test = train_test_split(
	X_scaled, y, test_size=0.2, random_state=RANDOM_STATE
)
logging.info(f'Tamanho do treino: {X_train.shape[0]} exemplos')
logging.info(f'Tamanho do teste: {X_test.shape[0]} exemplos')

# salvar o nome das features para uso futuro
feature_names = X.columns.tolist()
joblib.dump(feature_names, Path.cwd().parent / 'ml_models' / 'feature_names.joblib')
logging.info(f"Feature names salvo em: {Path.cwd().parent / 'ml_models' / 'feature_names.joblib'}")

2026-02-28 21:56:54,408 [INFO] [C:\Users\DELL\AppData\Local\Temp\ipykernel_17400\3842605674.py:4:<module>] Tamanho do treino: 924 exemplos
2026-02-28 21:56:54,408 [INFO] [C:\Users\DELL\AppData\Local\Temp\ipykernel_17400\3842605674.py:5:<module>] Tamanho do teste: 232 exemplos
2026-02-28 21:56:54,451 [INFO] [C:\Users\DELL\AppData\Local\Temp\ipykernel_17400\3842605674.py:10:<module>] Feature names salvo em: g:\Meu Drive\pascon_ofc\_fiap_tech_challenges\tech_challenge_fase5\ml_models\feature_names.joblib


# Treinamento do modelo

In [13]:
from fiap.utils.model_train import treinar_modelos

# =====================================================
# Modelos e grids de hiperparâmetros
# =====================================================
modelos = {
	'Regressão Linear': LinearRegression(),
	'Árvore de Decisão': DecisionTreeRegressor(random_state=RANDOM_STATE),
	'Random Forest': RandomForestRegressor(random_state=RANDOM_STATE, n_jobs=-1),
	'HistGradientBoosting': HistGradientBoostingRegressor(random_state=RANDOM_STATE),
	'XGBoost': XGBRegressor(
		random_state=RANDOM_STATE, n_jobs=-1, objective='reg:squarederror', verbosity=0
	),
}

param_grids = {
	'Regressão Linear': {'fit_intercept': [True, False], 'positive': [True, False]},
	'Árvore de Decisão': {
		'max_depth': [None, 5, 10, 15],
		'min_samples_split': [2, 5, 10],
		'min_samples_leaf': [1, 2, 4],
	},
	'Random Forest': {
		'n_estimators': [100, 200, 300],
		'max_depth': [None, 5, 10],
		'min_samples_split': [2, 5],
		'min_samples_leaf': [1, 2],
	},
	'HistGradientBoosting': {
		'max_iter': [100, 200],
		'max_depth': [None, 5, 10],
		'learning_rate': [0.01, 0.1, 0.2],
		'min_samples_leaf': [20, 50],
	},
	'XGBoost': {
		'n_estimators': [100, 200, 300],
		'max_depth': [3, 5, 7],
		'learning_rate': [0.01, 0.1, 0.2],
		'subsample': [0.8, 1.0],
		'colsample_bytree': [0.8, 1.0],
		'reg_alpha': [0, 0.1, 1],  # L1
		'reg_lambda': [1, 5, 10],  # L2
	},
}

df_resultados, melhor_modelo_geral = treinar_modelos(
	X_train=X_train,
	X_test=X_test,
	y_train=y_train,
	y_test=y_test,
	modelos=modelos,
	param_grids=param_grids,
	experiment_name='Defasagem Escolar - Modelos',
	model_dir=Path.cwd().parent / 'ml_models',
	random_state=RANDOM_STATE,
)

2026-02-28 21:56:54,476 [INFO] [G:\Meu Drive\pascon_ofc\_fiap_tech_challenges\tech_challenge_fase5\src\fiap\utils\model_train.py:41:treinar_modelos] ========== INÍCIO DO TREINAMENTO ==========
2026-02-28 21:56:54,832 [INFO] [G:\Meu Drive\pascon_ofc\_fiap_tech_challenges\tech_challenge_fase5\src\fiap\utils\model_train.py:56:treinar_modelos] Treinando modelo: Regressão Linear
2026-02-28 21:57:08,077 [INFO] [G:\Meu Drive\pascon_ofc\_fiap_tech_challenges\tech_challenge_fase5\src\fiap\utils\model_train.py:77:treinar_modelos] Modelo: Regressão Linear | MAE: 0.4082 | RMSE: 0.5538 | R²: 0.5605 | CV_MAE: 0.4269
c:\Users\DELL\AppData\Local\pypoetry\Cache\virtualenvs\tech-challenge-fase5-J-zMTxXs-py3.11\Lib\site-packages\mlflow\models\model.py:1209: FutureWarning: Saving scikit-learn models in the pickle or cloudpickle format requires exercising caution because these formats rely on Python's object serialization mechanism, which can execute arbitrary code during deserialization.The recommended sa

In [17]:
# Dados do melhor modelo geral
logging.info(melhor_modelo_geral)

2026-02-28 22:06:12,423 [INFO] [C:\Users\DELL\AppData\Local\Temp\ipykernel_17400\675946936.py:2:<module>] RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=345)


# Validação de extremos novamente para ver se o modelo está coerente

In [14]:
logging.info('Validando predições para alunos com defasagem extrema (-2 e 2)')
for val, idx_list in zip([-2, 2], [y[y == -2].index, y[y == 2].index]):
	if len(idx_list) > 0:
		i = idx_list[0]
		logging.info(f'Aluno com defasagem {val}: X={X.iloc[i].to_dict()}, y={y.iloc[i]}')
		logging.info(f'Predição do modelo: {melhor_modelo_geral.predict([X_scaled[i]])[0]:.4f}')

2026-02-28 22:03:08,752 [INFO] [C:\Users\DELL\AppData\Local\Temp\ipykernel_17400\3492575710.py:1:<module>] Validando predições para alunos com defasagem extrema (-2 e 2)
2026-02-28 22:03:08,759 [INFO] [C:\Users\DELL\AppData\Local\Temp\ipykernel_17400\3492575710.py:5:<module>] Aluno com defasagem -2: X={'fase': 0.0, 'idade': 10.0, 'iaa': 9.002, 'ieg': 8.6136363635, 'ips': 7.51, 'ipp': 7.1875, 'ida': 7.5, 'mat': 10.0, 'por': 5.0, 'ipv': 6.27, 'genero_f': 0.0, 'genero_m': 1.0, 'instituição_tipo_1': 1.0, 'instituição_tipo_2': 0.0, 'instituição_tipo_3': 0.0, 'instituição_tipo_4': 0.0, 'instituição_tipo_5': 0.0, 'instituição_tipo_6': 0.0, 'instituição_tipo_7': 0.0}, y=-2
2026-02-28 22:03:08,916 [INFO] [C:\Users\DELL\AppData\Local\Temp\ipykernel_17400\3492575710.py:6:<module>] Predição do modelo: -1.8367
2026-02-28 22:03:08,919 [INFO] [C:\Users\DELL\AppData\Local\Temp\ipykernel_17400\3492575710.py:5:<module>] Aluno com defasagem 2: X={'fase': 5.0, 'idade': 14.0, 'iaa': 9.168, 'ieg': 9.4318181